# Module: Dojo

This workflow...



In [3]:
from rush import build_blocking_provider

In [4]:
# |hide
# hidden setup for the notebook
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/benchmark_notebook").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR, exist_ok=True)
# swap into clean workdir so that our tests are deterministic
os.chdir(WORK_DIR)
PUT_YOUR_PREFERRED_WORKING_DIRECTORY_HERE = WORK_DIR
PUT_YOUR_TOKEN_HERE = os.environ["RUSH_TOKEN"]
RUSH_URL = os.environ["RUSH_URL"]
os.environ["RUSH_RESTORE_BY_DEFAULT"] = "False"

In [5]:
client = build_blocking_provider(
    access_token=PUT_YOUR_TOKEN_HERE,
    url = RUSH_URL,
    # for example, if your token is 00000000-dddd-cccc-0000-11111111,
    # then you should put access_token="00000000-dddd-cccc-0000-11111111"
    # (including the double quotes)
)

2025-02-06 16:20:37,527 - rush - INFO - Not restoring by default via env


In [9]:
benchmark = client.benchmark(name="OpenFF CDK2 RMSD17 Benchmark")

In [22]:
# |hide
from IPython.display import Markdown as md
rex_code_above = """
let

    auto3d = \\smi ->  map to_data (get 0 (auto3d_rex_s default_runspec_gpu { k = 1 } [smi])),

    dojo = \\default_runspec -> \\config -> \\input_trc -> \\input_ffxml ->
        dojo_rex_s default_runspec config input_trc input_ffxml

in
\\input ->
    let
        smol_id = id (get 1 input),
        smiles = smi (load smol_id "Smol"),
        smol_structure = auto3d smiles,
        
        dojo_conf = {
            simulation = {
                step_size_ps = 0.001,
                stages = [
                    {
                        Nvt = {
                            temperature_kelvin = 300,
                            steps = 1000
                        }
                    }
                ]
            },
            trajectory = {
                interval = 100,
                info = true
            }
        },

        dojo_out        = dojo      default_runspec dojo_conf   [smol_structure] []

    in
        []
"""

md(f"```haskell{rex_code_above}```")

```haskell
let

    auto3d = \smi ->  map to_data (get 0 (auto3d_rex_s default_runspec_gpu { k = 1 } [smi])),

    dojo = \default_runspec -> \config -> \input_trc -> \input_ffxml ->
        dojo_rex_s default_runspec config input_trc input_ffxml

in
\input ->
    let
        smol_id = id (get 1 input),
        smiles = smi (load smol_id "Smol"),

        dojo_conf = {
            simulation = {
                step_size_ps = 0.001,
                stages = [
                    {
                        Nvt = {
                            temperature_kelvin = 300,
                            steps = 1000
                        }
                    }
                ]
            },
            trajectory = {
                interval = 100,
                info = true
            }
        },

        dojo_out        = dojo      default_runspec dojo_conf   [smol_structure] []

    in
        []
```

In [23]:
submission = client.run_benchmark(
    benchmark.id, 
    rex_code_above, 
    "dojo", 
    sample=0.2)

View your submission at https://rush-qdx-2-staging.web.app/project/07c7d14a-3a55-491c-ab50-65cca07ec7a0/runs?selectedRunId=56c70920-8430-40a8-ac4c-4b00d27482cb
